## Imports

In [ ]:
import logging
import os
import sys
from io import StringIO
from pathlib import Path

import IPython.display as display
import requests
from IPython.display import HTML
from linkml_runtime.dumpers import json_dumper, yaml_dumper
from linkml_runtime.loaders import json_loader, yaml_loader
from PIL import Image

from omero_quay import test_path
from omero_quay.clients.http import get_manifest, post_request
from omero_quay.core.config import get_conf
from omero_quay.core.connect import omero_conn
from omero_quay.core.manifest import Manifest
from omero_quay.core.provenance import get_data_root
from omero_quay.parsers.excel import parse_xlsx

log = logging.getLogger("omero_quay")
log.setLevel("DEBUG")

## Settings


See [the documentation](../docs/configuration_file_content.md)







#### Configuration file path 

In [ ]:
test_path

In [ ]:
# path to the test configuration file

os.environ["QUAY_CONF"] = (
    (test_path / "containers/quay_docker_host.yml").resolve().as_posix()
)
print(
    f"Using configuration from {os.environ['QUAY_CONF']}",
)

#### Data Path settings

In [ ]:
conf = get_conf()
# local path to the test data
# also sets the path to the default excel file

test_data_path = Path(os.environ.get("QUAY_TEST_DATA", ".")).resolve()

conf["quay"]["DATA_PATH"] = test_data_path
conf["ingest"][
    "PROVENANCE_URL"
] = f"file://{Path('../tests/containers/provenance').resolve()}"


source_data_path = (test_data_path / "facility0" / "S-JCBD-200902093").resolve()
if source_data_path.exists():
    print(
        "Using data from ",
        source_data_path,
    )
else:
    print(f"{source_data_path} does not exist")

## Update and display the source data directory tree

In [ ]:
!cd { source_data_path } && tree -H . > source_data_tree.html

In [ ]:
# %load source_data_tree.html
with (source_data_path / "source_data_tree.html").open("r") as html:
    content = html.read().replace("Directory Tree", "Source Directory Tree")


HTML(content)

# Facility to docker route

## Putting the test data in the facility file system


On the `quay-facility` container, the test data directory is mounted on `/tmp/QuayTestData/`

We do an explicit copy here to avoid writing directly into the original test data on the host system. 

In a production setting, file copy is limited, except accross name space (provenance) boundaries.

In [ ]:
%%bash

# Copy data from  docker volume
docker exec quay-facility bash -c 'cp -R /tmp/QuayTestData/facility0 /mnt/user/home/'


In [ ]:
xlsx_path = test_data_path / "excels" / "test_JCB_facility.xlsx"
if not xlsx_path.exists():
    print("Bad path")
else:
    print(f"Using {xlsx_path.resolve()}")


manifest = parse_xlsx(xlsx_path, conf)

for assay in manifest.assays:
    print("- Assay: ", assay.name)
    for link in assay.importlinks:
        print("\t Import URL: ", link.srce_url)
print("\n | source - \n | ", end="")

for i, store in enumerate(manifest.route):
    data_root = get_data_root(manifest, store.id, store.scheme, template=True)
    print(f"{i} | {store.id} | {store.post_url} | {data_root} |", end="\n | ")
print("destination")

# Running the request to import data


Now the data is in the facility machine. We can't interact with it directly, but rather send a request to the quay service on the omero-server machine in the (simulated) mesocenter.

In [ ]:
manifest.step = 0
manifest_json = json_dumper.dumps(manifest)

manifest_id = post_request(manifest_json, conf=conf, post_url="http://localhost:8898")
print(f"Manifest submitted with id {manifest_id}")

# Tracking 

In [ ]:
import time

from datetime import datetime


def as_date(timestamp):
    date = datetime.fromisoformat(timestamp)
    return date.strftime("%H:%M:%S — %d/%m/%Y")


elapsed = 0
timeout = 600

prev_states = ()
current_states = ()
while elapsed < timeout:

    manifest = get_manifest(conf, manifest_id)
    if isinstance(manifest, int):
        print("manifest don't look right")
        print(manifest)
    elif manifest is None:
        print(".", end="\n")
        time.sleep(10)
        continue

    current_states = [state.status for state in manifest.states]
    if current_states == prev_states:
        print(".", end="\n")
        time.sleep(10)
        continue

    if current_states != prev_states:
        for state in manifest.states:
            print(as_date(state.timestamps[-1]))
            print(f"{state.id}: \t{state.status}")

        # clear cache
        prev_states = current_states

    current_states = (state.status for state in manifest.states)
    for state in manifest.states:
        if state == "errored":
            msg = f"Import failed for {state.id}"
            raise ValueError(msg)

    if set(current_states) == {"checked"}:
        print("** \\o/ **")

    if manifest.step == len(manifest.route) - 1:
        break

else:
    raise TimeoutError

In [ ]:
len(manifest.images)

In [ ]:
manifest.images[0].urls

In [ ]:
!docker exec quay-facility bash -c "ls /mnt/coop/JCB2009/Figure1"

In [ ]:
!docker exec -u irods irods-icat bash -c "ils -r /tempZone/home/JCB2009" 


### Retrieve a manifest from mongo with its ID.

Once the data is imported in the federated storage,  a manifest is created and stored to a mongo db by the clerk.


In [ ]:
stored_manifest = get_manifest(conf, manifest.id, conf["ingest"]["POST_URL"])
stored_manifest.id == manifest.id

In [ ]:
stored_manifest.timestamps

In [ ]:
len(stored_manifest.images)

In [ ]:
stored_manifest.images[0].ome_id